**STARTUP INVESTMENT**
*Venture deals, organizations, people and exits*

**PROBLEM DEFINITION**
> Tracking and analyzing investment trends over time

**DATA**
> This data is downloaded from kaggle notebook with the url
  https://www.kaggle.com/justinas/startup-investments
  
**CONTENT**
> This diverse dataset contains information about the startup ecosystem: organizations, individuals, company news, funding rounds, acquisitions, and IPOs.

> There are 11 tables that can be joined using unique IDs (schema to follow). More information about the individual data variables can be found on the Crunchabse Data website (under the API Entities Types section).

> No extensive data quality check have been performed yet. The information is available up to December 2013.

> While Kaggle contains other datasets focused on startup investments, to the best of my knowledge, this dataset has not yet been published on the platform and is unique.

**AKNOWLEDGEMENT**
> This Crunchbase 2013 Snapshot © 2013 dataset is fully attributed to Crunchbase.

**EVALUATION**
> Time Series would be used to focast


**INSPIRATIONS**
> There are multiple avenues for exploration:
EDA of the startup ecosystem.
Tracking and analyzing investment trends over time
Clustering VC funds based on their existing investments.
Predicting which startup will proceed to raise further rounds / will get acquired / will file for an IPO.
Mapping the network of individuals involved in the startup ecosystem.

In [ ]:
# Importing Libraries for Loading Dataset
import numpy as np
import pandas as pd

In [ ]:
# Importing required libraries for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from io import StringIO
from sklearn.ensemble import RandomForestRegressor

In [ ]:
objects = pd.read_csv("../input/startup-investments/objects.csv", low_memory=False)
objects.head()

In [ ]:
objects.dtypes

In [ ]:
objects['entity_type'].value_counts()

In [ ]:
objects['status'].value_counts()

In [ ]:
# Rename id in objects.csv to founded_object_id
objects.rename(columns={'id':'funded_object_id'}, inplace=True)
objects.head()

In [ ]:
objects.drop(["created_at","updated_at", "logo_url", "logo_width","overview", "entity_id","parent_id","normalized_name", "logo_height","short_description", "created_at", "updated_at", "twitter_username","relationships", "domain", "homepage_url", "overview", "tag_list","city", "region", "state_code"], axis="columns", inplace=True)
objects.info()

In [ ]:
objects['category_code'].value_counts()

In [ ]:
objects['country_code'].value_counts()

In [ ]:
investments = pd.read_csv("../input/startup-investments/investments.csv")
investments.head()

In [ ]:
investments['funded_object_id'].value_counts()

In [ ]:
# Loading and merging the required dataset

df = investments.merge(objects, on='funded_object_id')
df.head()

In [ ]:
df.info()

In [ ]:
df['status'].value_counts()

In [ ]:
df.drop(["closed_at", "first_investment_at","invested_companies", "investment_rounds", "created_at", "updated_at"], axis="columns", inplace= True)

In [ ]:
df.tail()

In [ ]:
df['funded_object_id'].value_counts()

In [ ]:
A = pd.read_csv("../input/startup-investments/funding_rounds.csv")
A.head()

In [ ]:
A.info()

In [ ]:
A.drop(['id', 'funding_round_id', 'funding_round_code', 'raised_amount', 'raised_currency_code', 'pre_money_valuation_usd', 'pre_money_valuation', 'pre_money_currency_code', 'post_money_valuation_usd', 'post_money_currency_code', 'participants', 'is_first_round', 'is_last_round', 'source_url', 'source_description', 'created_by', 'updated_at', 'created_at'], axis='columns', inplace=True)

In [ ]:
A.drop(['post_money_valuation'], axis='columns', inplace=True)

In [ ]:
A.rename(columns={'object_id':'funded_object_id'}, inplace=True)
A.head()

In [ ]:
df2 = df.merge(A, on='funded_object_id')
df2.head()

In [ ]:
df2.info()

In [ ]:
len(df2)

In [ ]:
df2.isna().sum()

In [ ]:
# Check in percentage the missing data
# summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(df2.isnull().sum()/len(df2.index)), 2)

In [ ]:
df2.drop(['created_by', 'first_milestone_at', 'last_milestone_at', 'last_investment_at'], axis='columns', inplace=True)

In [ ]:
df2.head()

In [ ]:
df2['category_code'].value_counts()

In [ ]:
# Check in percentage the missing data
# summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(df2.isnull().sum()/len(df2.index)), 2)

In [ ]:
#Dropping rows based on null columns
df2 = df2[~(df2['country_code'].isnull() | df2['description'].isnull() | df2['funded_at'].isnull() | df2['founded_at'].isnull())]

In [ ]:
df2.isna().sum()

In [ ]:
df2 = df2[~(df2['category_code'].isnull())]

In [ ]:
df2.isna().sum()

In [ ]:
df2['status'].value_counts()

In [ ]:
#Identify duplicates records in the data
dupes=df2.duplicated()
sum(dupes)

In [ ]:
df2=df2.drop_duplicates()

In [ ]:
#Identify duplicates records in the data
dupes2=df2.duplicated()
sum(dupes2)

# **LETS DO SOME ANALYSIS**

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
plt.rcParams['figure.figsize'] = 10,10
labels = df2['status'].value_counts().index.tolist()
sizes = df2['status'].value_counts().tolist()
explode = (0, 0.2, 0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)
ax1.axis('equal')
plt.title("What is start up companies current status", fontdict=None, position= [0.48,1], size = 'x-large')

plt.show()

Most of company (78.5 %) in this dataset is operating,

and around 3.0 % company is already closed.

In [ ]:
len(df['category_code'].unique())

In [ ]:
df2['category_code'].value_counts()[:5]

because we have around 43 categories of start up,

Then just plot all   : )

In [ ]:
plt.rcParams['figure.figsize'] = 15,8

height = df2['category_code'].value_counts()[:45].tolist()
bars =  df2['category_code'].value_counts()[:45].index.tolist()
y_pos = np.arange(len(bars))
plt.bar(y_pos, height , width=0.7 ,color= ['c']+['paleturquoise']*14)
plt.xticks(y_pos, bars)
plt.xticks(rotation=90)
plt.title("All Start-Up market category", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

**Now lets look at the top 15 startup category market**

In [ ]:
plt.rcParams['figure.figsize'] = 15,8

height = df2['category_code'].value_counts()[:15].tolist()
bars =  df2['category_code'].value_counts()[:15].index.tolist()
y_pos = np.arange(len(bars))
plt.bar(y_pos, height , width=0.7 ,color= ['c']+['paleturquoise']*14)
plt.xticks(y_pos, bars)
plt.xticks(rotation=90)
plt.title("Top 15 Start-Up market category", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

Lets do something interesting

In [ ]:
def count_word(df2, ref_col, liste):
    keyword_count = dict()
    for s in liste: keyword_count[s] = 0
    for liste_keywords in df[ref_col].str.split('|'):        
        if type(liste_keywords) == float and pd.isnull(liste_keywords): continue        
        for s in [s for s in liste_keywords if s in liste]: 
            if pd.notnull(s): keyword_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count


def makeCloud(Dict,name,color):
    words = dict()

    for s in Dict:
        words[s[0]] = s[1]

        wordcloud = WordCloud(width=1500, height=750, background_color=color, max_words=50, max_font_size=500, normalize_plurals=False)
        wordcloud.generate_from_frequencies(words)


    fig = plt.figure(figsize=(12, 8))
    plt.title(name)
    plt.imshow(wordcloud)
    plt.axis('off')

    plt.show()

In [ ]:
set_keywords = set()
for liste_keywords in df['category_code'].str.split('|').values:
    if isinstance(liste_keywords, float): continue  # only happen if liste_keywords = NaN
    set_keywords = set_keywords.union(liste_keywords)
#_________________________


In [ ]:
keyword_occurences, dum = count_word(df, 'category_code', set_keywords)

In [ ]:
makeCloud(keyword_occurences[0:15],"Keywords","White")

The most popular category is still about Software, Biotech & Web,

It maybe because these 3 categories are easily to scalable ? 

In [ ]:
df2.head()

**TOTAL FUNDING USD**

In [ ]:
df2['raised_amount_usd'].head()

In [ ]:
plt.rcParams['figure.figsize'] = 15,6
plt.hist(df2['raised_amount_usd'].dropna(), bins=30)
plt.ylabel('Count')
plt.xlabel('Fnding (usd)')
plt.title("Distribution of total funding ", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

In [ ]:
Q1 = df2['raised_amount_usd'].quantile(0.25)
Q3 = df2['raised_amount_usd'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = (Q1 - 1.5 * IQR)
upper_bound = (Q3 + 1.5 * IQR)

In [ ]:
without_outlier = df2[(df2['raised_amount_usd'] > lower_bound ) & (df2['funding_total_usd'] < upper_bound)]

In [ ]:
plt.rcParams['figure.figsize'] = 15,6
plt.hist(without_outlier['raised_amount_usd'].dropna(), bins=30,color = 'paleturquoise' )

plt.ylabel('Count')
plt.xlabel('Funding (usd)')
plt.title("Distribution of total funding ", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
df2['name'].value_counts()[:1000]

In [ ]:
plt.rcParams['figure.figsize'] = 15,8

height = df2['name'].value_counts()[:60].tolist()
bars =  df2['name'].value_counts()[:60].index.tolist()
y_pos = np.arange(len(bars))
plt.bar(y_pos, height , width=0.7 ,color= ['c']+['paleturquoise']*14)
plt.xticks(y_pos, bars)
plt.xticks(rotation=90)
plt.title("Top 60 Company names", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 15,8

height = df2['name'].value_counts()[:15].tolist()
bars =  df2['name'].value_counts()[:15].index.tolist()
y_pos = np.arange(len(bars))
plt.bar(y_pos, height , width=0.7 ,color= ['c']+['paleturquoise']*14)
plt.xticks(y_pos, bars)
plt.xticks(rotation=90)
plt.title("Top 15 Company names", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

In [ ]:
df2['founded_at'] = pd.to_datetime(df2['founded_at'], errors = 'coerce' )

In [ ]:
plt.rcParams['figure.figsize'] = 15,6
df2['name'].groupby(df2["founded_at"].dt.year).count().plot(kind="line")

plt.ylabel('Count')
plt.title("Founded distribution ", fontdict=None, position= [0.48,1.05], size = 'x-large')
plt.show()

In [ ]:
Facebook_founded_year = df2['founded_at'][df2['name']=="Facebook"].dt.year
Twitter_founded_year  = df2['founded_at'][df2['name']=="Twitter"].dt.year
Pinterest_founded_year  = df2['founded_at'][df2['name']=="Pinterest"].dt.year

In [ ]:
Facebook_founded_year

In [ ]:
Twitter_founded_year

In [ ]:
Pinterest_founded_year

 **Country Code**

In [ ]:
len(df2['country_code'].unique())

We have 94 unique code in the dataset

In [ ]:
df2['country_code'].value_counts()[:10]

Most of the companies came from USA

In [ ]:
df2['count'] = 1
country_market = df2[['count','country_code','category_code']].groupby(['country_code','category_code']).agg({'count': 'sum'})
# Change: groupby state_office and divide by sum
country_market_pct = country_market.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
country_market_pct.reset_index(inplace = True)

In [ ]:
USA_market_pct = country_market_pct[country_market_pct['country_code'] == "USA"]
USA_market_pct = USA_market_pct.sort_values('count',ascending = False)[0:10]

In [ ]:
## USA
plt.rcParams['figure.figsize'] =10,10
labels = list(USA_market_pct['category_code'])+['Other...']
sizes = list(USA_market_pct['count'])+[100-USA_market_pct['count'].sum()]
explode = (0.18, 0.12, 0.09,0,0,0,0,0,0,0,0.01)
colors =  ['royalblue','mediumaquamarine','moccasin'] +['oldlace']*8

plt.pie(sizes, explode = explode, colors = colors ,labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=30)
plt.axis('equal')
plt.tight_layout()
plt.title("USA start up market", fontdict=None, position= [0.48,1.1], size = 'x-large')

plt.show()
print('For USA, Most of start up market is about Software & Technology')

**Now let me check for my counry Nigeria**

In [ ]:
NGA_market_pct = country_market_pct[country_market_pct['country_code'] == "NGA"]
NGA_market_pct = NGA_market_pct.sort_values('count',ascending = False)[0:10]

In [ ]:
plt.rcParams['figure.figsize'] = 10,10
labels = list(NGA_market_pct['category_code'])+['Other...']
sizes = list(NGA_market_pct['count'])+[100-USA_market_pct['count'].sum()]
Nones = (0.18, 0.12, 0.09,0,0,0,0.01)
colors =  ['royalblue','violet','gold'] +['oldlace']*8

plt.pie(sizes, explode = Nones, colors = colors ,labels=labels, autopct='%1.1f%%',shadow=False, startangle=30)
plt.axis('equal')
plt.tight_layout()
plt.title("Nigeria start up market", fontdict=None, position= [0.48,1.1], size = 'x-large')
plt.show()
print('For NGA, Most of start up market is about Social mainly')

these two pie charts show how different of interest trend between Nigeria and America 

In [ ]:
df2.to_csv('EDA.csv', index=False, header=1)

# **Lets do some Time Series Analysis**

In [ ]:
# Import data again but this time parse dates
df3 = pd.read_csv("./EDA.csv",
                 low_memory=False,
                 parse_dates=["funded_at", "founded_at"])

In [ ]:
df3.head().T

In [ ]:
df3["saleYear"] = df3.funded_at.dt.year
df3["saleMonth"] = df3.funded_at.dt.month
df3["saleDay"] = df3.funded_at.dt.day
df3["saleDayOfWeek"] = df3.funded_at.dt.dayofweek
df3["saleDayOfYear"] = df3.funded_at.dt.dayofyear

In [ ]:
df3.head().T

In [ ]:
# Let's build a machine learning model 
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

model.fit(df3.drop("funding_total_usd", axis=1), df3["funding_total_usd"])

In [ ]:
# Find the columns which contain strings
for label, content in df3.items():
    if pd.api.types.is_string_dtype(content):
        print(label) 

In [ ]:
# This will turn all of the string value into category values
for label, content in df3.items():
    if pd.api.types.is_string_dtype(content):
        df3[label] = content.astype("category").cat.as_ordered()

In [ ]:
df3.info()

In [ ]:
# Check for columns which aren't numeric
for label, content in df3.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
# Turn categorical variables into numbers and fill missing
for label, content in df3.items():
    if not pd.api.types.is_numeric_dtype(content):
        # Add binary column to indicate whether sample had missing value
        df3[label+"_is_missing"] = pd.isnull(content)
        # Turn categories into numbers and add +1
        df3[label] = pd.Categorical(content).codes+1

In [ ]:
%%time
# Instantiate model 
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

model.fit(df3.drop("funding_total_usd", axis=1), df3["funding_total_usd"])

In [ ]:
df3['funding_total_usd'].value_counts()

In [ ]:
df3['raised_amount_usd'].value_counts()

In [ ]:
# Score the model
model.score(df3.drop("funding_total_usd", axis=1), df3["funding_total_usd"])

In [ ]:
df3.saleYear.value_counts()

**Split the data**

In [ ]:
# Split data into training and validation
df_val = df3
df_train = df3

len(df_val), len(df_train)

In [ ]:
# Split data into X & y
X_train, y_train = df_train.drop("funding_total_usd", axis=1), df_train.funding_total_usd
X_valid, y_valid = df_val.drop("funding_total_usd", axis=1), df_val.funding_total_usd

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
# Create evaluation function (the competition uses RMSLE)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

def rmsle(y_test, y_preds):
    """
    Caculates root mean squared log error between predictions and
    true labels.
    """
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate model on a few different levels
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": r2_score(y_train, train_preds),
              "Valid R^2": r2_score(y_valid, val_preds)}
    return scores

In [ ]:
%%time
# Cutting down on the max number of samples each estimator can see improves training time
model.fit(X_train, y_train)

In [ ]:
show_scores(model)